In [1]:
!pip install opendatasets --upgrade

In [2]:
import opendatasets as od
import os
import cv2
import random
import matplotlib.pyplot as plt
import pickle
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

In [3]:
od.download('https://www.kaggle.com/competitions/vision-verse/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: aksharsingh
Your Kaggle Key: ··········


100%|██████████| 1.98G/1.98G [00:14<00:00, 143MB/s]



Extracting archive ./vision-verse/vision-verse.zip to ./vision-verse


In [4]:
train_data_dir = '/content/vision-verse/data/train';
Categories = ['crownandrootrot','healthywheat','leafrust','wheatloosesmut']


In [ ]:
# import imageio as iio
# count=0
# for c in Categories:
#   dirr=train_data_dir+"/"+c
#   for l in os.listdir(dirr):
#     imgg=iio.imread(dirr+"/"+l)
#     if l[-1]=='f' and l[-2]=='i' and l[-3]=='g':
#       print(l)


00571.gif
07071.gif
00541.gif
00281.gif
00531.gif
03681.gif
00021.gif
00511.gif
00431.gif


KeyboardInterrupt: ignored

In [ ]:
# import imageio as iio
# count=0

# for c in Categories:
#   dirr=train_data_dir+"/"+c
#   for l in os.listdir(dirr):
#     imgg=iio.imread(dirr+"/"+l)
#     imgg=np.resize(imgg, (32, 32, 3))
#     data.append([imgg,Categories.index(c)])
  


In [ ]:
# data=np.array(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [5]:
IMG_SIZE = 224
data=[]
i=0
for category in Categories:
  folder = os.path.join(train_data_dir,category)
  label = Categories.index(category)
  for img in os.listdir(folder):
    img_path = os.path.join(folder,img)
    img_arr = cv2.imread(img_path)
   # print(img_arr)
    # try:
    #        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    #        print(img.shape)
    # except:
    #     print('err')
    # print(category,i)
    try:
        # image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_AREA)
        img_arr = cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
        # print(img_arr.shape)
    except:
        print(img_arr,2)
        continue
    print(img_path)
    img_arr = cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
    data.append([img_arr,label])
    i=i+1

/content/vision-verse/data/train/crownandrootrot/03671.png
/content/vision-verse/data/train/crownandrootrot/06351.jpg
/content/vision-verse/data/train/crownandrootrot/03411.jpg
/content/vision-verse/data/train/crownandrootrot/07741.jpg
/content/vision-verse/data/train/crownandrootrot/08441.jpg
/content/vision-verse/data/train/crownandrootrot/03151.jpg
/content/vision-verse/data/train/crownandrootrot/03341.jpg
/content/vision-verse/data/train/crownandrootrot/06481.jpg
/content/vision-verse/data/train/crownandrootrot/02591.jpg
/content/vision-verse/data/train/crownandrootrot/09161.jpg
/content/vision-verse/data/train/crownandrootrot/07541.jpg
/content/vision-verse/data/train/crownandrootrot/03821.jpg
/content/vision-verse/data/train/crownandrootrot/06331.jpg
/content/vision-verse/data/train/crownandrootrot/00061.jpg
/content/vision-verse/data/train/crownandrootrot/09941.jpg
/content/vision-verse/data/train/crownandrootrot/09501.jpg
/content/vision-verse/data/train/crownandrootrot/03561.j

In [6]:
random.shuffle(data)
batch_size = 32

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    shuffle = True,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    shuffle = True,
    subset='validation') # set as validation data

Found 1813 images belonging to 4 classes.
Found 451 images belonging to 4 classes.


In [8]:
train_generator


In [9]:
feature = []
label = []

for features,labels in data:
  feature.append(features)
  label.append(labels)
feature = np.array(feature)
label = np.array(label)
feature = feature.astype('float')
feature=feature/255

feature.shape

(2543, 224, 224, 3)

In [10]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [11]:

import tensorflow as tf


model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu',input_shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.BatchNormalization(axis=-1),

    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.BatchNormalization(axis=-1),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.BatchNormalization() ,
    tf.keras.layers.Dropout(rate=0.5),

    tf.keras.layers.Dense(4,activation='softmax')

])
learning_rate = 0.001
epochs=2
opt= tf.keras.optimizers.Adam(learning_rate=learning_rate , decay=learning_rate/(epochs*0.5))
model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [12]:
model.fit(feature, label, epochs=10, validation_split=0.1)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


72/72 [==============================] - 26s 192ms/step - loss: 1.7403 - accuracy: 0.4943 - val_loss: 1.1581 - val_accuracy: 0.4745
Epoch 2/10
72/72 [==============================] - 11s 151ms/step - loss: 0.9496 - accuracy: 0.6429 - val_loss: 1.2343 - val_accuracy: 0.5059
Epoch 3/10
72/72 [==============================] - 11s 151ms/step - loss: 0.6636 - accuracy: 0.7605 - val_loss: 1.1945 - val_accuracy: 0.5137
Epoch 4/10
72/72 [==============================] - 11s 151ms/step - loss: 0.3833 - accuracy: 0.8733 - val_loss: 1.1476 - val_accuracy: 0.4863
Epoch 5/10
72/72 [==============================] - 11s 152ms/step - loss: 0.2794 - accuracy: 0.9082 - val_loss: 0.7854 - val_accuracy: 0.7059
Epoch 6/10
72/72 [==============================] - 11s 152ms/step - loss: 0.1763 - accuracy: 0.9423 - val_loss: 0.9547 - val_accuracy: 0.5843
Epoch 7/10
72/72 [==============================] - 11s 153ms/step - loss: 0.1183 - accuracy: 0.9677 - val_loss: 0.4468 - val_accuracy: 0.8510
Epoch 8/10

In [ ]:
dir = '/content/vision-verse/data/test'
df = pd.read_csv('/content/vision-verse/data/sample_submission.csv')
print(df)

                       path            label
0      data/test/000146.jpg  crownandrootrot
1      data/test/000191.jpg  crownandrootrot
2      data/test/000185.jpg  crownandrootrot
3      data/test/000218.jpg  crownandrootrot
4      data/test/000224.jpg  crownandrootrot
...                     ...              ...
1096   data/test/000995.jpg  crownandrootrot
1097  data/test/0001058.jpg  crownandrootrot
1098   data/test/000981.jpg  crownandrootrot
1099  data/test/0001064.jpg  crownandrootrot
1100  data/test/0001070.jpg  crownandrootrot

[1101 rows x 2 columns]


In [ ]:
testing = []
c=0
count =0
for img in os.listdir(dir):
    img_path = os.path.join(dir,img)
    img_arr = cv2.imread(img_path)
    try:
        # image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_AREA)
        img_arr = cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
        # img_arr = cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
        # 'crownandrootrot','healthywheat','leafrust','wheatloosesmut']
        img_arr=img_arr/255
        img_arr = np.expand_dims(img_arr,axis=0)
        val = model.predict(img_arr)
        if(val[0][0]>=val[0][1] and val[0][0]>=val[0][2] and val[0][0]>=val[0][3]):
          df.loc[df['path']=='data/test/'+str(img),label2]='crownandrootrot'
        elif(val[0][1]>=val[0][0] and val[0][1]>=val[0][2] and val[0][1]>=val[0][3]):
          df.loc[df['path']=='data/test/'+str(img),label2]='healthywheat'
        elif(val[0][2]>=val[0][0] and val[0][2]>=val[0][1] and val[0][2]>=val[0][3]):
          df.loc[df['path']=='data/test/'+str(img),label2]='leafrust'
        elif(val[0][3]>=val[0][0] and val[0][3]>=val[0][2] and val[0][3]>=val[0][1]):
          df.loc[df['path']=='data/test/'+str(img),label2]='wheatloosesmut'
        count=count+1
        print(count)
        # print(img_arr.shape)
    except:
        df.loc[df['path']=='data/test/'+str(img),label]='crownandrootrot'
        c=c+1
        print(c)
        continue
    # img_arr = cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
    # img_arr=img_arr/255
    # img_arr = np.expand_dims(img_arr,axis=0)
    # val = model.predict(img_arr)
    # c=c+1
    # print(c)
    # print(val)
    # if(val[0][0]>val[0][1]):
    #   df.loc[df['path']=='data/test/' + str(img),label]='fake'
    # else:
    #   df.loc[df['path']=='data/test/' + str(img),label]='real'

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


KeyboardInterrupt: ignored

In [ ]:
df.to_csv('result2.csv', index=False)